In [1]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from typing import Optional, List, Tuple, Dict

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [3]:
class DS(Dataset):
    def __init__(self, X: np.ndarray, Y: np.ndarray):
        self.length = len(X)
        self.X = X
        self.Y = Y

    def __getitem__(self, idx):
        x = self.X[idx, :]
        y = self.Y[idx]
        return (x, y)

    def __len__(self):
        return self.length

In [4]:
# Hyperparameters
indim = 10
outdim = 2
hidden_layer = 100
lr = 0.01
batch_size = 64
epochs = 200

In [5]:
# Dataloader for training and test datasets

Xtrain = np.loadtxt("/Users/kantg/OneDrive/Desktop/CMU/Neural Networks and Deep learning/data/XTrain.txt", delimiter="\t")
Ytrain = np.loadtxt("/Users/kantg/OneDrive/Desktop/CMU/Neural Networks and Deep learning/data/yTrain.txt", delimiter="\t").astype(int)
Ytrain = torch.from_numpy(Ytrain)
Ytrain = Ytrain.type(torch.LongTensor)
m1, n1 = Xtrain.shape
print(m1, n1)
train_ds = DS(Xtrain, Ytrain)
train_loader = DataLoader(train_ds, batch_size=batch_size)

Xtest = np.loadtxt("/Users/kantg/OneDrive/Desktop/CMU/Neural Networks and Deep learning/data/XTest.txt", delimiter="\t")
Ytest = np.loadtxt("/Users/kantg/OneDrive/Desktop/CMU/Neural Networks and Deep learning/data/yTest.txt", delimiter="\t").astype(int)
Ytest = torch.from_numpy(Ytest)
Ytest = Ytest.type(torch.LongTensor)
m2, n2 = Xtest.shape
print(m2, n2)
test_ds = DS(Xtest, Ytest)
test_loader = DataLoader(test_ds, batch_size=batch_size)

500 10
200 10


In [6]:
class SingleLayerMLP(nn.Module):
    """constructing a single layer neural network with Pytorch"""
    def __init__(self, indim, outdim, hidden_layer):
        super(SingleLayerMLP, self).__init__()
        self.fc1 = nn.Linear(indim, hidden_layer)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_layer, outdim)


    def forward(self, x):
        x = x.to(torch.float32)
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [7]:
# Initialize the network

model = SingleLayerMLP(indim, outdim, hidden_layer).to(device)
print(model)

SingleLayerMLP(
  (fc1): Linear(in_features=10, out_features=100, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=100, out_features=2, bias=True)
)


In [8]:
# Loss and optimizer

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

In [9]:
# Training the model

def train(train_loader, model, loss_function, optimizer):
    size = len(train_loader.dataset)
    for batch, (X, y) in enumerate(train_loader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_function(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_size % 100 == 0:
            loss, current = loss.item(), (batch_size + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
# Validation using the test dataset 

def validate(test_loader, model, loss_function):
    size = len(test_loader.dataset)
    num_batches = len(test_loader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in test_loader:
            pred = model(X)
            test_loss += loss_function(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n") 


In [10]:
for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train(train_loader, model, loss_function, optimizer)
    validate(test_loader, model, loss_function)

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 91.5%, Avg loss: 0.249075 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 93.5%, Avg loss: 0.172940 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 94.0%, Avg loss: 0.154210 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 94.5%, Avg loss: 0.150129 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 93.5%, Avg loss: 0.149370 

Epoch 6
-------------------------------
Test Error: 
 Accuracy: 93.5%, Avg loss: 0.148486 

Epoch 7
-------------------------------
Test Error: 
 Accuracy: 94.5%, Avg loss: 0.145583 

Epoch 8
-------------------------------
Test Error: 
 Accuracy: 94.5%, Avg loss: 0.145257 

Epoch 9
-------------------------------
Test Error: 
 Accuracy: 94.0%, Avg loss: 0.145052 

Epoch 10
-------------------------------
Test Error: 
 Accuracy: 93.5%, Avg loss: 0.144081 

Epoch 11
-------------------------------
Test Error: 
 Accuracy: 93.5%, Avg los